In [23]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def MesoNet(input_shape):
    model = tf.keras.Sequential([
        # Block 1
        Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        # Block 2
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        # Block 3
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        # Block 4
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        # Flatten the convolutional features
        Flatten(),
        # Fully connected layers
        Dense(512, activation='relu'),
        Dense(1, activation='sigmoid')  # Output layer, sigmoid for binary classification
    ])
    return model

# Example usage:
input_shape = (256, 256, 3)  # Example input shape for images of size 256x256 with 3 channels (RGB)
meso_model = MesoNet(input_shape)


In [24]:
meso_model.save('meso_model.h5')

In [25]:
meso_model.compile(optimizer='adam',
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

In [20]:
# Train the model
history = meso_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs
)

# Evaluate the model
loss, accuracy = meso_model.evaluate(train_generator)
print('Training loss:', loss)
print('Training accuracy:', accuracy)

Epoch 1/10
222/222 [==============================] - 171s 762ms/step - loss: 0.2833 - accuracy: 0.8787
Epoch 2/10
222/222 [==============================] - 641s 3s/step - loss: 0.0881 - accuracy: 0.9657
Epoch 3/10
222/222 [==============================] - 139s 627ms/step - loss: 0.0544 - accuracy: 0.9811
Epoch 4/10
222/222 [==============================] - 144s 649ms/step - loss: 0.0427 - accuracy: 0.9873
Epoch 5/10
222/222 [==============================] - 142s 640ms/step - loss: 0.0232 - accuracy: 0.9918
Epoch 6/10
222/222 [==============================] - 165s 742ms/step - loss: 0.0178 - accuracy: 0.9932
Epoch 7/10
222/222 [==============================] - 164s 740ms/step - loss: 0.0310 - accuracy: 0.9900
Epoch 8/10
222/222 [==============================] - 164s 738ms/step - loss: 0.0089 - accuracy: 0.9963
Epoch 9/10
222/222 [==============================] - 139s 624ms/step - loss: 0.0025 - accuracy: 0.9992
Epoch 10/10
222/222 [==============================] - 28s 127ms/st

In [26]:
import tensorflow as tf
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array

# Load pre-trained MesoNet model
meso_model_path = 'meso_model.h5'
meso_model = tf.keras.models.load_model(meso_model_path)

# Function to classify image
def classify_image(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path)
    image = image.resize((256, 256))  # Resize image to match input shape
    image_array = img_to_array(image) / 255.0  # Convert image to array and normalize
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension

    # Classify image using MesoNet
    prediction = model.predict(image_array)[0][0]
    
    # Determine if it's a deepfake or not
    if prediction < 0.5:
        return "Real"
    else:
        return "Deepfake"

# Example usage
image_path = 'Meloni deepfake.jpg'  # Replace with the path to your image
result = classify_image(image_path, meso_model)
print("Prediction:", result)



1/1 [==============================] - 0s 79ms/step
Prediction: Deepfake


In [22]:
input_shape = (256, 256, 3)
batch_size = 32
epochs = 10

In [3]:
# Data loading and preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    './data/',
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='binary'
)

Found 7104 images belonging to 2 classes.


In [30]:
import streamlit as st
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np

# Load pre-trained MesoNet model
meso_model_path = 'meso_model.h5'
meso_model = tf.keras.models.load_model(meso_model_path)

# Function to classify image
def classify_image(image):
    # Preprocess image
    image = image.resize((256, 256))  # Resize image to match input shape
    image_array = img_to_array(image) / 255.0  # Convert image to array and normalize
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension

    # Classify image using MesoNet
    prediction = meso_model.predict(image_array)[0][0]
    
    # Determine if it's a deepfake or not
    if prediction < 0.5:
        return "Real"
    else:
        return "Deepfake"

# Streamlit interface
st.title('Deepfake Detection')

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "png"])

if uploaded_file is not None:
    # Display the uploaded image
    image = Image.open(uploaded_file)
    st.image(use_column_width=True)

    # Classify the image
    result = classify_image(image)
    st.write("Prediction:", result)
